In [1]:
!pip install sagemaker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 111.6 MB/s eta 0:00:00
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.131
    Uninstalling botocore-1.34.131:
      Successfully uninstalled botocore-1.34.131
  Attempting uninstall: boto3
    Found existing installation: boto3 1.34.131
    Uninstalling boto3-1.34.131:
      Successfully uninstalled boto3-1.34.131
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.1.1 requires nvidia-ml-py3==7.352.0, which is not installed.
aiobotocore 2.13.2 requires botocore<1.34.132,>=1.34.70, but you have botocore 1.35.28 which is incompatible.
amazon-sagemaker-sql-magic 0.1.3 requires sqlparse==0.5.0, but you have sqlparse 0.5.1 which is incompatible.
autogluon-core 1.1.1 requires scikit-learn<1.4.1,>=1.3.0, but you have scikit-learn 1.4.2 which is incompatible.
au

In [2]:
import sagemaker
import boto3

sess=sagemaker.Session()

sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    sagemaker_session_bucket=sess.default_bucket

try:
    role=sagemaker.get_execution_role()
except ValueError:
    iam=boto3.client('iam')
    role=iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess=sagemaker.Session(default_bucket=sagemaker_session_bucket)
print(f"sage maker role arn: {role}")
print(f"sagemaker session region:{sess.boto_region_name}")


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sage maker role arn: arn:aws:iam::608862829472:role/service-role/AmazonSageMaker-ExecutionRole-20240927T091375
sagemaker session region:ap-south-1


In [3]:
from sagemaker.huggingface import get_huggingface_llm_image_uri

# retrieve the llm image uri
llm_image = get_huggingface_llm_image_uri(
  "huggingface",
  version="0.8.2"
)

# print ecr image uri
print(f"llm image uri: {llm_image}")

llm image uri: 763104351884.dkr.ecr.ap-south-1.amazonaws.com/huggingface-pytorch-tgi-inference:2.0.0-tgi0.8.2-gpu-py39-cu118-ubuntu20.04


In [12]:
import sagemaker
role = sagemaker.get_execution_role()
print(role)


arn:aws:iam::608862829472:role/service-role/AmazonSageMaker-ExecutionRole-20240927T091375


In [8]:
from sagemaker.huggingface.model import HuggingFaceModel

# Define the Hugging Face model ID and task
hub = {
    'HF_MODEL_ID': 'distilbert-base-cased-distilled-squad',  # Correct model ID
    'HF_TASK': 'question-answering'
}

# Create HuggingFaceModel instance
huggingface_model = HuggingFaceModel(
    env=hub,
    role=role,
    transformers_version='4.26',
    pytorch_version='1.13',
    py_version='py39'
)

# Deploy the model to an ml.t3.medium instance
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large'
)

# Define input data for prediction
data = {
    "inputs": {
        "question": "What is used for inference?",
        "context": "This model is used with SageMaker to answer the questions."
    }
}

# Perform prediction using the deployed model
result = predictor.predict(data)

# Print the prediction result
print(result)


------!{'score': 0.13097743690013885, 'start': 24, 'end': 33, 'answer': 'SageMaker'}


In [15]:
#For classification
import sagemaker
role = sagemaker.get_execution_role()
hub = {
    'HF_MODEL_ID': 'distilbert-base-uncased-finetuned-sst-2-english',
    'HF_TASK': 'text-classification'
}
# Create HuggingFaceModel instance
huggingface_model = HuggingFaceModel(
    env=hub,
    role=role,
    transformers_version='4.26',
    pytorch_version='1.13',
    py_version='py39'
)
# Deploy the model to an ml.t3.medium instance
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large'
)
# Input data for text classification
data = {
    "inputs": "I love this movie, it is awesome!"
}

# Perform prediction
result = predictor.predict(data)
print(result)

------![{'label': 'POSITIVE', 'score': 0.999879002571106}]


In [28]:
# text generation
from sagemaker.huggingface.model import HuggingFaceModel

hub = {
    'HF_MODEL_ID': 'distilgpt2',
    'HF_TASK': 'text-generation'
}

role = 'nn' #the role created in IAM with SageMaker Full Access

huggingface_model = HuggingFaceModel(
    env=hub,
    role='nn',
    transformers_version='4.26',
    pytorch_version='1.13',
    py_version='py39'
)

predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large'
)

data = {
    "inputs": "The future of AI is"
}

result = predictor.predict(data)
print(result)


------![{'generated_text': 'The future of AI is now in our hands." They are confident that a solution will come within 10 years, he said.\n\n\n\n"Some of the key points in technology could be changed. There will be good, more efficient AI development'}]
